In [1]:
#EDITED CODE FROM Q3
# adapted from lecture 2 demo code kmeans_digit.py
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import show_image
from scipy.sparse import csc_matrix, find
from sklearn.cluster import KMeans
import time
#kmeans algorithm

def kmeans_alg(pixels,k):
    #pixels - input image 
    #k number of desired clusters

    matFile = sio.loadmat(pixels)
    #print(matFile)
    x = matFile['xtest']/255
    #y = matFile['ytest']
    
    m = x.shape[1]
    #print(x.shape)
    
    #STEP1: RANDOMLY INITALIZED CENTROIDS
    np.random.seed(3)
    #c= np.random.randint(0,255,size= [k,3]) --> this only works for 'football.bmp' for some reason
    #https://stackoverflow.com/questions/63337505/assigning-random-centroids-by-index
    indx = np.random.choice(x.shape[0], k, replace=False)
    c = x[indx]
    #print(c)
    #print(c.shape)
    
    
    iterno = 200
    #count= 0
    for iter in range(0, iterno):
        #count = count +1
        #print("--iteration %d \n" % iter)

        #STEP 2: DECIDE CLUSTER MEMBERSHIPS
        #CALC L2-NORM 
        #https://stackoverflow.com/questions/4370975/python-numpy-euclidean-distance-calculation-between-matrices-of-row-vectors
        #https://numpy.org/doc/stable/user/basics.broadcasting.html helped resolve broadcasting error with np.newaxis
        #print(c[ :, np.newaxis].shape)
        dist = (x - c[ :, np.newaxis]) ** 2
        dist = np.sum(dist, axis=2) #adjusted for axis=2 since newaxis added a column in the middle
        dist = np.sqrt(dist)
        #print(dist.shape)
        
        #assign to nearest centroid- similar to kmeans_digit file except switched back to argmin here
        labels= np.argmin(dist, axis=0)
        #print(labels)
        
        #STEP 3: ADJUST CLUSTERS
        #https://dev.to/sajal2692/coding-k-means-clustering-using-python-and-numpy-fg1
        new_c = np.empty((k, m))
        for i in range(k):
            new_c[i] = np.mean(x[labels ==i], axis=0)
        #print(new_c)
        
        #STEP4: DID THE CLUSTERS MOVE?
        #https://stackoverflow.com/questions/45583472/how-to-find-the-difference-between-two-matrices-in-python-with-the-result-should
        diff= np.abs(new_c - c)
        #print(diff)
        #error comment suggested I use np.all or np.any
        if not np.any(diff):
            break
            
        c=new_c
    #print('# of iterations:{}'.format(count))
    return labels, new_c

In [2]:
pixels= 'mnist_10digits.mat'
k=10

pred_labels, pred_centroids= kmeans_alg(pixels,k)
#print(pred_labels.shape)

In [3]:
#purity score calc
from sklearn.metrics import confusion_matrix

matFile = sio.loadmat(pixels)
y = matFile['ytest']
#https://stackoverflow.com/questions/17869840/numpy-vector-n-1-dimension-n-dimension-conversion
y = np.squeeze(y)

pscore=[]
for i in range(k):
    
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
    c_matrix = confusion_matrix(y[y==i], pred_labels[y==i])
     
    #https://stackoverflow.com/questions/34047540/python-clustering-purity-metric
    i_pscore = np.sum(np.amax(c_matrix, axis=1)) / np.sum(c_matrix)
    pscore.append(i_pscore)
    print("for {} clusters" .format(i+1))
    print("the purity score is {}" .format(i_pscore))

#print(pscore)   


for 1 clusters
the purity score is 0.7887755102040817
for 2 clusters
the purity score is 0.5638766519823789
for 3 clusters
the purity score is 0.6589147286821705
for 4 clusters
the purity score is 0.7128712871287128
for 5 clusters
the purity score is 0.5570264765784114
for 6 clusters
the purity score is 0.32847533632286996
for 7 clusters
the purity score is 0.6722338204592901
for 8 clusters
the purity score is 0.6371595330739299
for 9 clusters
the purity score is 0.6560574948665298
for 10 clusters
the purity score is 0.5213082259663032


In [4]:
#Manhattan Distance
def kmeans_alg_l1(pixels,k):
    #pixels - input image 
    #k number of desired clusters

    matFile = sio.loadmat(pixels)
    #print(matFile)
    x = matFile['xtest']/255
    #y = matFile['ytest']
    
    m = x.shape[1]
    #print(x.shape)
    
    #STEP1: RANDOMLY INITALIZED CENTROIDS
    np.random.seed(3)
    #c= np.random.randint(0,255,size= [k,3]) --> this only works for 'football.bmp' for some reason
    #https://stackoverflow.com/questions/63337505/assigning-random-centroids-by-index
    indx = np.random.choice(x.shape[0], k, replace=False)
    c = x[indx]
    #print(c)
    #print(c.shape)
    
    
    iterno = 200
    #count= 0
    for iter in range(0, iterno):
        #count = count +1
        #print("--iteration %d \n" % iter)

        #CALC L1-NORM 
        #https://numpy.org/doc/stable/user/basics.broadcasting.html helped resolve broadcasting error with np.newaxis
        #print(c[ :, np.newaxis].shape)
        dist = np.abs(x - c[ :, np.newaxis])
        dist = np.sum(dist, axis=2)
        #print(dist.shape)
        
        #assign to nearest centroid- similar to kmeans_digit file except switched back to argmin here
        labels= np.argmin(dist, axis=0)
        #print(labels)
        
        #STEP 3: ADJUST CLUSTERS
        #https://dev.to/sajal2692/coding-k-means-clustering-using-python-and-numpy-fg1
        new_c = np.empty((k, m))
        for i in range(k):
            new_c[i] = np.median(x[labels ==i], axis=0)
        #print(new_c)
        
        #STEP4: DID THE CLUSTERS MOVE?
        #https://stackoverflow.com/questions/45583472/how-to-find-the-difference-between-two-matrices-in-python-with-the-result-should
        diff= np.abs(new_c - c)
        #print(diff)
        #error comment suggested I use np.all or np.any
        if not np.any(diff):
            break
            
        c=new_c
    #print('# of iterations:{}'.format(count))
    return labels, new_c

In [5]:
pred_labels_l1, pred_centroids= kmeans_alg_l1(pixels,k)
#print(pred_labels_l1.shape)

pscore=[]
for i in range(k):
    
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
    c_matrix = confusion_matrix(y[y==i], pred_labels_l1[y==i])
     
    #https://stackoverflow.com/questions/34047540/python-clustering-purity-metric
    i_pscore = np.sum(np.amax(c_matrix, axis=1)) / np.sum(c_matrix)

    pscore.append(i_pscore)
    print("for {} clusters" .format(i+1))
    print("the purity score is {}" .format(i_pscore))
    
#print(pscore)

for 1 clusters
the purity score is 0.7061224489795919
for 2 clusters
the purity score is 0.5444933920704845
for 3 clusters
the purity score is 0.5
for 4 clusters
the purity score is 0.6524752475247525
for 5 clusters
the purity score is 0.5376782077393075
for 6 clusters
the purity score is 0.2937219730941704
for 7 clusters
the purity score is 0.6680584551148225
for 8 clusters
the purity score is 0.6439688715953308
for 9 clusters
the purity score is 0.5379876796714579
for 10 clusters
the purity score is 0.4697720515361744


In [6]:
# on average it looks like eucleadian distance has a better purity score compared to L1